
#  torch.nn.Module vs torch.nn.functional

When you're building neural networks in PyTorch, you have two main ways to define operations:

| Aspect | **torch.nn.Module** | **torch.nn.functional** |
|:--|:--|:--|
| **What** | Defines *layers* (objects) that keep track of parameters and states. | Defines *functions* (stateless operations) you can apply directly to tensors. |
| **Parameters** | Automatically holds and manages parameters like weights and biases. | You must **manually** pass parameters like weights. |
| **Use Case** | When building a model class (`nn.Module`) — like building blocks. | When you want **fine-grained control** inside `forward()` without creating a separate module. |
| **Examples** | `nn.Conv2d`, `nn.ReLU`, `nn.Linear` | `F.conv2d`, `F.relu`, `F.linear` |
| **Good For** | Standard model definition, easier saving/loading, readable code. | Custom forward passes, experimental models. |

---

#  **torch.nn.Module examples**

These are **objects**.  
When you create them, they *automatically create parameters* inside, and you just call them.

```python
import torch
import torch.nn as nn

# Create a tensor
x = torch.randn(1, 3, 32, 32)  # (batch, channels, height, width)

# Conv2d layer
conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
out = conv(x)

# Activation
relu = nn.ReLU()
out = relu(out)

# Linear (fully connected) layer
fc = nn.Linear(16 * 32 * 32, 10)  # flatten first
out = out.view(out.size(0), -1)  # flatten
out = fc(out)

# Softmax
softmax = nn.Softmax(dim=1)
out = softmax(out)

print(out.shape)  # should be (batch_size, 10)
```

Notice:  
- `nn.Conv2d`, `nn.Linear` automatically create weights and biases.
- You *instantiate* them once, and use them.
- Parameters are stored inside the objects (`conv.weight`, `conv.bias`).

---

#  **torch.nn.functional examples**

These are **functions**.  
You must **manually** provide inputs and parameters (sometimes).

```python
import torch
import torch.nn.functional as F

# Create tensor
x = torch.randn(1, 3, 32, 32)

# Define weights manually
weight = torch.randn(16, 3, 3, 3, requires_grad=True)
bias = torch.randn(16, requires_grad=True)

# Convolution (you must pass weights yourself!)
out = F.conv2d(x, weight=weight, bias=bias, stride=1, padding=1)

# Activation
out = F.relu(out)

# Flatten
out = out.view(out.size(0), -1)

# Define manual weights for linear layer
linear_weight = torch.randn(10, 16 * 32 * 32, requires_grad=True)
linear_bias = torch.randn(10, requires_grad=True)

# Linear layer
out = F.linear(out, weight=linear_weight, bias=linear_bias)

# Softmax
out = F.softmax(out, dim=1)

print(out.shape)
```

✅ Notice:
- `F.conv2d`, `F.linear` need explicit `weight` and `bias`.
- It's **low-level** — you have **full control**.
- Useful for building custom layers or experimenting.

---

#  One more simple comparison:

Suppose you want a simple ReLU:

```python
# With nn.Module
relu_layer = nn.ReLU()
out = relu_layer(x)

# With nn.functional
out = F.relu(x)
```
Here for ReLU, it's almost the same — but **Conv2d**, **Linear** are very different because they have parameters.

---

#  In Short:

| If you need | Use |
|:--|:--|
| Layers with parameters | `torch.nn.Module` |
| Simple operations without parameters | `torch.nn.functional` |
| Fine control over forward | `torch.nn.functional` |
| Easy readable model | `torch.nn.Module` |

---
